In [5]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm


In [6]:
# Define the directory containing the images
image_directory = 'data/test/'

# Define the path to the .h5 model
model_path = 'cancer_model_v01.h5'

# Load the model
model = K.models.load_model(model_path)



In [8]:
# Initialize lists to store predictions and image filenames
predictions = []
image_filenames = []

# Get the total number of TIFF images
total_images = len(os.listdir(image_directory))

# Iterate over the TIFF images in the directory
for filename in tqdm(os.listdir(image_directory), total=total_images, desc='Processing images'):
    if filename.endswith('.tiff') or filename.endswith('.tif'):
        # Load the image
        image_path = os.path.join(image_directory, filename)
        image = load_img(image_path, target_size=(32, 32))
        image_array = img_to_array(image)
        image_array = image_array / 255.0  # Normalize the image

        # Expand dimensions to match the model input shape
        image_array = tf.expand_dims(image_array, 0)

        # Perform the prediction
        prediction = model.predict(image_array, verbose=0)[0]
        predicted_label = tf.argmax(prediction).numpy()

        # Append the prediction and image filename to the lists
        predictions.append(predicted_label)
        image_filenames.append(filename)

Processing images: 100%|██████████| 57458/57458 [1:36:17<00:00,  9.94it/s]


'b'

In [13]:
# Create a dataframe with the image filenames and predictions
data = {'id': image_filenames, 'label': predictions}
df = pd.DataFrame(data)
df["id"] = df["id"].apply(lambda x: x.replace(".tif",""))

In [14]:
# Save the dataframe to a CSV file
csv_file = 'output_predictions.csv'
df.to_csv(csv_file, index=False)